# Pandas

To recap, this is Python:
https://docs.python.org/3/index.html

For today, this is Pandas:
https://pandas.pydata.org/docs/

and this is the Pandas tutorial:
https://pandas.pydata.org/docs/user_guide/10min.html

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [ ]:
penguins = sns.load_dataset('penguins')

In [ ]:
penguins.head()

Arrays in numpy (and pandas) are in row-major order. This means that when you're indexing, you should put in the row name or number and then the column name or number. When you get the shape of a dataframe, it will put the number of rows before the number of columns:

In [ ]:
penguins.shape

In [ ]:
print("Number of rows:", penguins.shape[0])
print("Number of columns:", penguins.shape[1])

In [ ]:
penguins['species']

In [ ]:
penguins.iloc[0, 1]

In [ ]:
penguins.iloc[1, 0]

In [ ]:
penguins.loc[2, 'body_mass_g']

In [ ]:
penguins.loc[0]

In [ ]:
penguins.loc[0, ['species', 'island']]

In [ ]:
penguins.iloc[0, 0:2]

In [ ]:
penguins.iloc[0, :2]

In [ ]:
penguins.iloc[0, :]

In [ ]:
penguins.iloc[40:50, :]

In [ ]:
penguins.head(50).tail(10)

In [ ]:
penguins.describe()

In [ ]:
penguins.dropna(how='all')

In [ ]:
penguins.dropna(how='any')

In [ ]:
penguins.dropna(subset=penguins.columns[:-1], how='any')

In [ ]:
penguins.groupby('sex').mean()

In [ ]:
penguins.groupby('sex').agg(['mean', 'std', 'median'])

In [ ]:
penguins.groupby(['species', 'island']).count()

In [ ]:
penguins.groupby(['island', 'species']).count()

In [ ]:
penguins.groupby(['species', 'island']).mean()

In [ ]:
penguins[penguins['species'] == 'Gentoo']

In [ ]:
penguins['species'] == 'Gentoo'

In [ ]:
adelies = penguins[penguins['species'] == 'Gentoo']
adelies.head()

In [ ]:
species_filter = penguins['species'] == 'Adelie'
island_filter = penguins['island'] == 'Dream'
dream_adelies = penguins[species_filter & island_filter]
dream_adelies.groupby(['species', 'island']).count()

In [ ]:
dream_or_adelies = penguins[species_filter | island_filter]
dream_or_adelies.groupby(['species', 'island']).count()

In [ ]:
cleaned = penguins.dropna(how='any').copy()

In [ ]:
cleaned['bill_surface_mm2'] = cleaned['bill_length_mm'] * cleaned['bill_depth_mm'] * 3.14159
cleaned.head()

## Penguin BMI

Penguin BMI is an important indicator of health. The formula, as we all know, is body mass divided by the product of bill length and flipper length. Use the code cell below to calculate the average (mean) BMI of Adelie penguins on the island of Dream.

### Some quick plots

In [ ]:
penguins.plot?

In [ ]:
penguins.plot(x='bill_length_mm', y='bill_depth_mm', kind='scatter')

In [ ]:
penguins.plot(y='bill_length_mm', kind='kde')

In [ ]:
sns.regplot(data=penguins, x='bill_length_mm', y='bill_depth_mm')

In [ ]:
sns.lmplot(data=penguins, x='bill_length_mm', y='bill_depth_mm', hue='species')

## An Excel interlude

In [ ]:
box = pd.read_excel('data/box.xlsx', sheet_name = 'Box1', header=None)
box.head()

In [ ]:
box.melt()

In [ ]:
box.to_numpy().flatten()

In [ ]:
box.reset_index()

In [ ]:
box.reset_index().melt(id_vars='index')

In [ ]:
box.melt(ignore_index=False).reset_index()

In [ ]:
longform = box.melt(ignore_index=False).reset_index()
longform

In [ ]:
longform.columns = ['Row', 'Column', 'Sample ID']
longform['Row'] += 1
longform['Column'] += 1
longform.head()

In [ ]:
box_indexed = pd.read_excel('data/box.xlsx', sheet_name = 'Box1 (labeled)')
box_indexed.head()

In [ ]:
serum_box_10 = box_indexed.melt(id_vars='Serum Box 10').rename(columns={'Serum Box 10': 'Row', 'variable': 'Column', 'value': 'Sample ID'})
serum_box_10.head()

## Boldly reading from Wikipedia

Take a look at this article:
https://en.wikipedia.org/wiki/List_of_largest_cities

Wikipedia can be a quick source of tables, and you can also get output in the form of html tables from a number of different online tools. However, they tend to be a little messier:

In [ ]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_largest_cities")
city_df = dfs[1]
city_df.head()

You can see that this has two levels of columns, they're named a littile strangely, it seems to have pulled in the citations and footnotes in brackets, and it starts with a totally empty row. All of this requires some degree of cleaning, but just avoiding the copy and paste step can be kind of nice.

In [ ]:
city_df.columns = city_df.columns.map(' | '.join)
city_df = city_df.dropna(how='all')
city_df.head()

Column names are a little weird, but at least we have just one row of columns. You can copy and paste the column names directly from the output into your code:

In [ ]:
city_df = city_df.set_index('City[a] | City[a]')
city_df

Any time you have a column with all unique values, you can use them as an index. This lets you do very quick lookups:

In [ ]:
city_df.loc['Manila', :]

And you can also grab multiple cities:

In [ ]:
city_df.loc[['Delhi', 'Manila', 'Yangon'], :]

These aren't exactly meaningful statistics, but they are quick! Any time you're using a lot of functions, it can help readability to put them on their own line. In Python, in order to stop the interpreter from complaining you have to wrap the whole expression in an extra set of parentheses:

In [ ]:
(city_df.groupby('Country | Country')
        .agg(['count', 'mean', 'median'])
        .sort_values(by=('UN 2018 population estimates[b] | UN 2018 population estimates[b]', 'count'), ascending=False)
        .head())

In [ ]:
# You can politely ignore this data cleaning
plot_df = city_df[(city_df['Urban area[8] | Area(km2)'] != '—')].loc[:, ['Urban area[8] | Area(km2)', 'Urban area[8] | Population']].astype(float)
# This is a neat renaming
plot_df.columns = ['Area(km2)', 'Population']
sns.scatterplot(data=plot_df, x='Area(km2)', y='Population')
plt.show()
print()
print("What's the outlier?")
print()
print(plot_df[plot_df['Area(km2)'] > 10000])
print()
print("It was us! Filter it out:")
print()
sns.scatterplot(data=plot_df[plot_df['Area(km2)'] < 10000], x='Area(km2)', y='Population')
plt.show()
sns.regplot(data=plot_df[plot_df['Area(km2)'] < 10000], x='Area(km2)', y='Population')
plt.show()